In [1]:
import pandas as pd
import numpy as np
from builddataset import build_dataset
from raw_processing import label_schemes
import pickle

### Build dataset

In [2]:
# Load dataset
lc_date = '08102023'
cvs_df = pd.read_csv('../processed_data/AAVSOCVsZTFxmatch_21032023_BTS_manual_labelled.csv')
cvs_df = label_schemes(cvs_df)
print(cvs_df.shape, cvs_df.columns)

# Extract features
features = build_dataset(cvs_df, objcol='oid_ztf', folderpath=f'../lightcurves_dataset/lasair_{lc_date}', outliercap=False)


(5708, 16) Index(['oid_ztf', 'oid_aavso', 'type_aavso', 'ra', 'dec', 'Eclipsing',
       'CV_Types', 'CV_subtypes', 'CV_subsubtypes', 'eclipse_clear',
       'manual_label', 'Clarity', 'labels_1', 'labels_2', 'labels_3',
       'labels_4'],
      dtype='object')
INFO: Login to gaia TAP server [astroquery.gaia.core]
OK
INFO: Login to gaia data server [astroquery.gaia.core]
OK
500 Error 500:
Can not execute the following SQL: 
DROP TABLE user_dmistry.ast_table
Because: ERROR: table "ast_table" does not exist
500 Error 500:
Can not execute the following SQL: 
DROP TABLE user_dmistry.xmatch_table
Because: ERROR: table "xmatch_table" does not exist
Sending pytable.
Uploaded table 'ast_table'.
Retrieving table 'user_dmistry.ast_table'
Table 'user_dmistry.ast_table' updated.
INFO: Query finished. [astroquery.utils.tap.core]
Table 'ast_table' deleted.
Removed jobs: '['1696768756944O']'.
INFO: Gaia TAP server logout OK [astroquery.gaia.core]
INFO: Gaia data server logout OK [astroquery.gaia.cor

In [3]:
dataset_lcfeats = features[0]
dataset_metafeats = features[1]
dataset_allfeats = features[2]
outlier_thresholds = features[3]

save = True
cvlistdate = '21032023'
lightcurves_date = '08102023'

# Save datasets
if save == True:
    dataset_lcfeats.to_csv(f'../processed_data/dataset{cvlistdate}_lcfeatures{lightcurves_date}.csv', index=False)
    dataset_metafeats.to_csv(f'../processed_data/dataset{cvlistdate}_gaiafeatures{lightcurves_date}.csv', index=False)
    dataset_allfeats.to_csv(f'../processed_data/dataset{cvlistdate}_allfeatures{lightcurves_date}.csv', index=False)
    with open(f'../processed_data/outlier_thresholds{cvlistdate}_{lightcurves_date}.pkl', 'wb') as f:
        pickle.dump(outlier_thresholds, f)

In [4]:
dataset_inc_labels = pd.merge(cvs_df[['oid_ztf', 'oid_aavso', 'type_aavso', 'Eclipsing',
                           'CV_Types', 'CV_subtypes', 'CV_subsubtypes', 'eclipse_clear',
                           'manual_label', 'Clarity', 'labels_1', 'labels_2', 'labels_3',
                           'labels_4']],
                           dataset_allfeats, on='oid_ztf', how='left')

if save == True:
    dataset_inc_labels.to_csv(f'../processed_data/dataset_{cvlistdate}allfeatures_inc_labels{lightcurves_date}.csv', index=False)

In [5]:
data = pd.read_csv(f'../processed_data/dataset_{cvlistdate}allfeatures_inc_labels{lightcurves_date}.csv')
# identify mixed dtype columns
wierd_cols = []
for col in data.columns:
    weird = (data[[col]].applymap(type) != data[[col]].iloc[0].apply(type)).any(axis=1)
    if len(data[weird]) > 0:
        wierd_cols.append(col)
        print(col)

# convert floats to strings for the weird columns
for col in wierd_cols:
    data[col] = data[col].astype(str)


data.to_csv(f'../processed_data/dataset_{cvlistdate}allfeatures_inc_labels{lightcurves_date}.csv', index=False)


oid_aavso
CV_Types
CV_subtypes
CV_subsubtypes
manual_label
labels_1
labels_2
labels_3
labels_4


In [7]:
pd.options.display.max_columns = None
data

,oid_ztf,oid_aavso,type_aavso,Eclipsing,CV_Types,CV_subtypes,CV_subsubtypes,eclipse_clear,manual_label,Clarity,labels_1,labels_2,labels_3,labels_4,Amplitude_g,AndersonDarling_g,Autocor_length_g,Beyond1Std_g,CAR_mean_g,CAR_sigma_g,CAR_tau_g,Con_g,Eta_e_g,FluxPercentileRatioMid20_g,FluxPercentileRatioMid35_g,FluxPercentileRatioMid50_g,FluxPercentileRatioMid65_g,FluxPercentileRatioMid80_g,Freq1_harmonics_amplitude_0_g,Freq1_harmonics_amplitude_1_g,Freq1_harmonics_amplitude_2_g,Freq1_harmonics_amplitude_3_g,Freq1_harmonics_rel_phase_1_g,Freq1_harmonics_rel_phase_2_g,Freq1_harmonics_rel_phase_3_g,Freq2_harmonics_amplitude_0_g,Freq2_harmonics_amplitude_1_g,Freq2_harmonics_amplitude_2_g,Freq2_harmonics_amplitude_3_g,Freq2_harmonics_rel_phase_1_g,Freq2_harmonics_rel_phase_2_g,Freq2_harmonics_rel_phase_3_g,Freq3_harmonics_amplitude_0_g,Freq3_harmonics_amplitude_1_g,Freq3_harmonics_amplitude_2_g,Freq3_harmonics_amplitude_3_g,Freq3_harmonics_rel_phase_1_g,Freq3_harmonics_rel_phase_2_g,Freq3_harmonics_rel_phase_3_g,Gskew_g,LinearTrend_g,MaxSlope_g,Mean_g,Meanvariance_g,MedianAbsDev_g,MedianBRP_g,PairSlopeTrend_g,PercentAmplitude_g,PercentDifferenceFluxPercentile_g,PeriodLS_g,Period_fit_g,Psi_CS_g,Psi_eta_g,Q31_g,Rcs_g,Skew_g,SlottedA_length_g,SmallKurtosis_g,Std_g,StetsonK_g,StetsonK_AC_g,StructureFunction_index_21_g,StructureFunction_index_31_g,StructureFunction_index_32_g,Amplitude_r,AndersonDarling_r,Autocor_length_r,Beyond1Std_r,CAR_mean_r,CAR_sigma_r,CAR_tau_r,Con_r,Eta_e_r,FluxPercentileRatioMid20_r,FluxPercentileRatioMid35_r,FluxPercentileRatioMid50_r,FluxPercentileRatioMid65_r,FluxPercentileRatioMid80_r,Freq1_harmonics_amplitude_0_r,Freq1_harmonics_amplitude_1_r,Freq1_harmonics_amplitude_2_r,Freq1_harmonics_amplitude_3_r,Freq1_harmonics_rel_phase_1_r,Freq1_harmonics_rel_phase_2_r,Freq1_harmonics_rel_phase_3_r,Freq2_harmonics_amplitude_0_r,Freq2_harmonics_amplitude_1_r,Freq2_harmonics_amplitude_2_r,Freq2_harmonics_amplitude_3_r,Freq2_harmonics_rel_phase_1_r,Freq2_harmonics_rel_phase_2_r,Freq2_harmonics_rel_phase_3_r,Freq3_harmonics_amplitude_0_r,Freq3_harmonics_amplitude_1_r,Freq3_harmonics_amplitude_2_r,Freq3_harmonics_amplitude_3_r,Freq3_harmonics_rel_phase_1_r,Freq3_harmonics_rel_phase_2_r,Freq3_harmonics_rel_phase_3_r,Gskew_r,LinearTrend_r,MaxSlope_r,Mean_r,Meanvariance_r,MedianAbsDev_r,MedianBRP_r,PairSlopeTrend_r,PercentAmplitude_r,PercentDifferenceFluxPercentile_r,PeriodLS_r,Period_fit_r,Psi_CS_r,Psi_eta_r,Q31_r,Rcs_r,Skew_r,SlottedA_length_r,SmallKurtosis_r,Std_r,StetsonK_r,StetsonK_AC_r,StructureFunction_index_21_r,StructureFunction_index_31_r,StructureFunction_index_32_r,Q31_color,StetsonJ,StetsonL,median_g,min_mag_g,max_mag_g,n_obs_g,dif_min_mean_g,dif_min_median_g,dif_max_mean_g,dif_max_median_g,dif_max_min_g,temporal_baseline_g,kurtosis_g,pwr_max_g,freq_pwr_max_g,FalseAlarm_prob_g,pwr_maxovermean_g,npeaks_pt5to1_g,rrate_pt5to1_g,drate_pt5to1_g,amp_pt5to1_g,npeaks_1to2_g,rrate_1to2_g,drate_1to2_g,amp_1to2_g,npeaks_2to5_g,rrate_2to5_g,drate_2to5_g,amp_2to5_g,npeaks_above5_g,rrate_above5_g,drate_above5_g,amp_above5_g,rollstd_ratio_t20s10_g,stdstilllev_t20s10_g,rollstd_ratio_t10s5_g,stdstilllev_t10s5g,pnts_leq_rollMedWin20-1mag_g,pnts_leq_rollMedWin20-2mag_g,pnts_leq_rollMedWin20-5mag_g,pnts_geq_rollMedWin20+1mag_g,pnts_geq_rollMedWin20+2mag_g,pnts_geq_rollMedWin20+3mag_g,pnts_leq_median-1mag_g,pnts_leq_median-2mag_g,pnts_leq_median-5mag_g,pnts_geq_median+1mag_g,pnts_geq_median+2mag_g,pnts_geq_median+3mag_g,median_r,min_mag_r,max_mag_r,n_obs_r,dif_min_mean_r,dif_min_median_r,dif_max_mean_r,dif_max_median_r,dif_max_min_r,temporal_baseline_r,kurtosis_r,pwr_max_r,freq_pwr_max_r,FalseAlarm_prob_r,pwr_maxovermean_r,npeaks_pt5to1_r,rrate_pt5to1_r,drate_pt5to1_r,amp_pt5to1_r,npeaks_1to2_r,rrate_1to2_r,drate_1to2_r,amp_1to2_r,npeaks_2to5_r,rrate_2to5_r,drate_2to5_r,amp_2to5_r,npeaks_above5_r,rrate_above5_r,drate_above5_r,amp_above5_r,rollstd_ratio_t20s10_r,stdstilllev_t20s10_r,rollstd_ratio_t10s5_r,stdstilllev_t10s5r,pnts_leq_roll